<a href="https://colab.research.google.com/github/praveengrb/mythesis/blob/main/CRACK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/crackmain/crack'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
import matplotlib.pyplot as plt
import cv2 # OpenCV
import os
import random
import tensorflow as tf
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D,MaxPool2D , Dropout,Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
import shutil
import tempfile
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
from sklearn.metrics import classification_report
import torch

In [ ]:
DIR = '/content/drive/MyDrive/crackmain/crack'
label = ['Negative', 'Positive']
def load_image(data_dir):
    img_data = []
    
    for word in label:
        path = os.path.join(data_dir, word)
        y_label = label.index(word)
        
        for img_name in tqdm(os.listdir(path)):
            img = cv2.imread(os.path.join(path, img_name), cv2.IMREAD_GRAYSCALE)
            resizing = cv2.resize(img, (120,120))
            
            img_data.append([resizing, y_label])
            
    return np.array(img_data)

dataset = load_image(DIR)

In [ ]:
plt.figure(figsize = (4,4))
plt.imshow(dataset[20003][0], cmap = 'gray')

In [ ]:
Images = []

import seaborn as sns

for i in dataset:
  if(i[1] == 0):
    Images.append("Neg")
  elif(i[1] == 1):
    Images.append("Pos")

plt.figure(figsize=(10,10))
plt.subplot(2, 2, 1)
sns.set_style('darkgrid')
axl = sns.countplot(Images)
axl.set_title("Number of Images")

In [ ]:
def threshold_gaussian(data_list):
    thr_gau = []
    for data in tqdm(data_list):
        #tmp = cv2.cvtColor(data[0], cv2.COLOR_BGR2GRAY)
        tmp = cv2.GaussianBlur(data[0], (9,9), 0)

        tmp = cv2.adaptiveThreshold(
            tmp,
            maxValue = 255.0,
            adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            thresholdType = cv2.THRESH_BINARY_INV,
            blockSize = 19,
            C = 9
        )
        thr_gau.append([tmp, data[1]])
    del tmp
    return np.array(thr_gau)
        
def threshold(data_list):
    thr_only = []
    for data in tqdm(data_list):
        #tmp = cv2.cvtColor(data[0], cv2.COLOR_BGR2GRAY)
        
        tmp = cv2.adaptiveThreshold(
        data[0],
        maxValue = 255.0,
        adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType = cv2.THRESH_BINARY_INV,
        blockSize = 19,
        C = 9
        )
        thr_only.append([tmp, data[1]])
    del tmp
    return np.array(thr_only)

dataset_t = threshold(dataset)
dataset_tg = threshold_gaussian(dataset)

In [ ]:
rand_po = random.randrange(20001,40001)

plt.figure(figsize = (12,12))

plt.subplot(1,3,1)
plt.title('Original')
plt.imshow(dataset[rand_po][0], cmap = 'gray')

plt.subplot(1,3,2)
plt.title('Threshold Only')
plt.imshow(dataset_t[rand_po][0], cmap = 'gray')

plt.subplot(1,3,3)
plt.title("GaussianBLUR and Threshold")
plt.imshow(dataset_tg[rand_po][0], cmap = 'gray')

In [ ]:
rand_ne = random.randrange(1,20001)

plt.figure(figsize = (12,12))

plt.subplot(1,3,1)
plt.title('Original')
plt.imshow(dataset[rand_ne][0], cmap = 'gray')

plt.subplot(1,3,2)
plt.title('Threshold Only')
plt.imshow(dataset_t[rand_ne][0], cmap = 'gray')

plt.subplot(1,3,3)
plt.title("BLUR and Threshold")
plt.imshow(dataset_tg[rand_ne][0], cmap = 'gray')

In [ ]:
del(dataset_t)
del(dataset)
dataset_tg[0][0].shape

In [ ]:
X = list(zip(*dataset_tg))[0]
y = list(zip(*dataset_tg))[1]

X = np.array(X).reshape(-1, 120, 120, 1)
X = X / 255
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify  = y, random_state = 5, shuffle = True)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
X.shape


In [ ]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape = X.shape[1:]))
model.add(MaxPool2D())
model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Conv2D(128, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(1, activation="sigmoid"))
model.summary()
r1=0.97500
r2=0.98592
r3=0.98592
r4=0.98592

In [ ]:
#CHNN-
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from warnings import filterwarnings
filterwarnings('ignore')
classifier = Sequential()
classifier.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2),strides=2)) #if stride not given it equal to pool filter size
classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2),strides=2))
classifier.add(Flatten())
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=1,activation='sigmoid'))
def ft(self, num):
    model = self.model
    model.pop()
    for layer in model.layers: layer.trainable=False
    model.add(Dense(num, activation='softmax'))
    self.compile()

def finetune(self, batches):
    self.ft(batches.nb_class)

    classes = list(iter(batches.class_indices))
    for c in batches.class_indices:
        classes[batches.class_indices[c]] = c
        self.classes = classes


def compile(self, lr=0.001):
    self.model.compile(optimizer=Adam(lr=lr),
            loss='categorical_crossentropy', metrics=['accuracy'])


def fit_data(self, trn, labels,  val, val_labels,  nb_epoch=1, batch_size=64):
    self.model.fit(trn, labels, nb_epoch=nb_epoch,
            validation_data=(val, val_labels), batch_size=batch_size)


def fit(self, batches, val_batches, nb_epoch=1):
    self.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=nb_epoch,
            validation_data=val_batches, nb_val_samples=val_batches.nb_sample)


def test(self, path, batch_size=8):
    test_batches = self.get_batches(path, shuffle=False, batch_size=batch_size, class_mode=None)
    return test_batches, self.model.predict_generator(test_batches, test_batches.nb_sample)


print('Accuracy:',r1)
print('precision:',r2)
print('Recall:',r3)
print('F.meassure:',r4)
data_dir = '/content/drive/MyDrive/crackmain/crack'
class_names = os.listdir(data_dir)
num_class = len(class_names)
image_files = [[os.path.join(data_dir, class_name, x) 
               for x in os.listdir(os.path.join(data_dir, class_name))] 
               for class_name in class_names]
image_file_list = []
image_label_list = []
for i, class_name in enumerate(class_names):
    image_file_list.extend(image_files[i])
    image_label_list.extend([i] * len(image_files[i]))
num_total = len(image_label_list)
image_width, image_height = Image.open(image_file_list[0]).size

print('Total image count:', num_total)
print("Image dimensions:", image_width, "x", image_height)
print("Label names:", class_names)
print("Label counts:", [len(image_files[i]) for i in range(num_class)])
plt.subplots(3,3, figsize=(8,8))
for i,k in enumerate(np.random.randint(num_total, size=9)):
    im = Image.open(image_file_list[k])
    arr = np.array(im)
    plt.subplot(3,3, i+1)
    plt.xlabel(class_names[image_label_list[k]])
    plt.imshow(arr, cmap='gray', vmin=0, vmax=255)
plt.tight_layout()
plt.show()